In [30]:
import numpy as np
import cv2

#Todo:

#find grid for feet holds (two grids for the feet)
#find thresholds for each type

In [31]:
hand_x = [54,114,173,233,293,352,412,471,530,590,650,]
hand_y = [176,235,295,354,414,473,533,592,652,711,771,831,890,950,1009,1069,1128,1188,1247]

#need foot1 grid, foot 2 grid, bottom foot row coordinates

In [32]:
img = cv2.resize(cv2.imread('first.png'), (0, 0), fx=1, fy=1)
hand = cv2.resize(cv2.imread('hand.png'), (0, 0), fx=1, fy=1)
foot = cv2.resize(cv2.imread('foot.png'), (0, 0), fx=1, fy=1)
start = cv2.resize(cv2.imread('start.png'), (0, 0), fx=1, fy=1)
finish = cv2.resize(cv2.imread('finish.png'), (0, 0), fx=1, fy=1)

In [33]:
# zoom = 1
# img = cv2.resize(img, (0, 0), fx=zoom, fy=zoom)
# cv2.imshow('img', img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [34]:
grid = np.meshgrid(hand_x,hand_y)
grid = np.array(grid)
holds = grid.reshape(2,-1).T

In [35]:
def find_id(input_cord, holds = holds):
  x_dist = (input_cord[0] - holds[:, 0]) #All rows, 0th element in each row
  y_dist = (input_cord[1] - holds[:, 1]) #All rows, first element in each row
  distance = (x_dist**2) + (y_dist**2) #diagonal distance
  return np.argmin(distance)

def find_many_id(my_holds, holds = holds):
  return [find_id(my_holds[i]) for i in range(len(my_holds))]

In [36]:
hand_result = cv2.matchTemplate(img, hand, cv2.TM_CCOEFF_NORMED)
foot_result = cv2.matchTemplate(img, foot, cv2.TM_CCOEFF_NORMED)
start_result = cv2.matchTemplate(img, start, cv2.TM_CCOEFF_NORMED)
finish_result = cv2.matchTemplate(img, finish, cv2.TM_CCOEFF_NORMED)

In [37]:
hand_min_val, hand_max_val, hand_min_loc, hand_max_loc = cv2.minMaxLoc(hand_result)
foot_min_val, foot_max_val, foot_min_loc, foot_max_loc = cv2.minMaxLoc(foot_result)
start_min_val, start_max_val, start_min_loc, start_max_loc = cv2.minMaxLoc(start_result)
finish_min_val, finish_max_val, finish_min_loc, finish_max_loc = cv2.minMaxLoc(finish_result)

In [38]:
hand_threshold = .40
foot_threshold = .40
start_threshold = .50
finish_threshold = .50

In [39]:
hand_yloc, hand_xloc = np.where(hand_result >= hand_threshold)
foot_yloc, foot_xloc = np.where(foot_result >= foot_threshold)
start_yloc, start_xloc = np.where(start_result >= start_threshold)
finish_yloc, finish_xloc = np.where(finish_result >= finish_threshold)

In [40]:
hold_length = 60  
hold_height = 60  

In [41]:
hand_rectangles = [[x, y, hold_length, hold_height] for x,y  in zip(hand_xloc, hand_yloc)]

In [42]:
# hand_rectangles = []
# for (x, y) in zip(hand_xloc, hand_yloc):
#     hand_rectangles.append([x, y, hold_length, hold_height])

In [43]:
foot_rectangles = []
for (x, y) in zip(foot_xloc, foot_yloc):
    foot_rectangles.append([x, y, hold_length, hold_height])

In [44]:
start_rectangles = []
for (x, y) in zip(start_xloc, start_yloc):
    start_rectangles.append([x, y, hold_length, hold_height])

In [45]:
finish_rectangles = []
for (x, y) in zip(finish_xloc, finish_yloc):
    finish_rectangles.append([x, y, hold_length, hold_height])

In [46]:
hand_rectangles, weights = cv2.groupRectangles(hand_rectangles, 1, 0.2)
foot_rectangles, weights = cv2.groupRectangles(foot_rectangles, 1, 0.2)
start_rectangles, weights = cv2.groupRectangles(start_rectangles, 1, 0.2)
finish_rectangles, weights = cv2.groupRectangles(finish_rectangles, 1, 0.2)

In [47]:
for (x, y, w, h) in hand_rectangles:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,0), 2)

In [48]:
for (x, y, w, h) in foot_rectangles:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,0), 2)

In [49]:
for (x, y, w, h) in start_rectangles:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,0), 2)

In [50]:
for (x, y, w, h) in finish_rectangles:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,0), 2)

In [51]:
main_grid = [ [0]*11 for i in range(19)] #19 rows 11 columns

In [52]:
buffer = 5

test_grid = [ [0]*11 for i in range(19)] #19 rows 11 columns
for x in range(19):
    for y in range(11):
        top_left = (hand_x[y] - buffer, hand_y[x] - buffer)
        bottom_right = (hand_x[y] + buffer, hand_y[x] + buffer)
        
        test_grid[x][y] = (top_left, bottom_right)
        cv2.rectangle(img, top_left, bottom_right, (209,203,193), 2)
        
        
        for (x_pos, y_pos, _, _) in hand_rectangles:
            hand_position = (x_pos, y_pos)
            
            if(hand_position[0] >= top_left[0] and hand_position[0] <= bottom_right[0]#top left corner within grid
               and hand_position[1] >= top_left[1] and hand_position[1] <= bottom_right[1]): 
                cv2.circle(img, (x_pos, y_pos), 3, (0,0,0), 2)
                cv2.circle(img, (x_pos, y_pos), 3, (255,255,0), -1)
                main_grid[x][y] = 1
                #print(x_pos,y_pos)
        
        for (x_pos, y_pos, _, _) in foot_rectangles:
            foot_position = (x_pos, y_pos)
            
            if(hand_position[0] >= top_left[0] and hand_position[0] <= bottom_right[0]#top left corner within grid
               and hand_position[1] >= top_left[1] and hand_position[1] <= bottom_right[1]): 
                cv2.circle(img, (x_pos, y_pos), 3, (0,0,0), 2)
                cv2.circle(img, (x_pos, y_pos), 3, (255,170,0), -1)
                main_grid[x][y] = 1
                #print(x_pos,y_pos)
        
        for (x_pos, y_pos, _, _) in start_rectangles:
            start_position = (x_pos, y_pos)
            
            if(hand_position[0] >= top_left[0] and hand_position[0] <= bottom_right[0]#top left corner within grid
               and hand_position[1] >= top_left[1] and hand_position[1] <= bottom_right[1]): 
                cv2.circle(img, (x_pos, y_pos), 3, (0,0,0), 2)
                cv2.circle(img, (x_pos, y_pos), 3, (0,255,0), -1)
                main_grid[x][y] = 1
                #print(x_pos,y_pos)
        
        for (x_pos, y_pos, _, _) in finish_rectangles:
            finish_position = (x_pos, y_pos)
            
            if(hand_position[0] >= top_left[0] and hand_position[0] <= bottom_right[0]#top left corner within grid
               and hand_position[1] >= top_left[1] and hand_position[1] <= bottom_right[1]): 
                cv2.circle(img, (x_pos, y_pos), 3, (0,0,0), 2)
                cv2.circle(img, (x_pos, y_pos), 3, (255,0,255), -1)
                main_grid[x][y] = 1
                #print(x_pos,y_pos)
        

In [53]:
zoom = 0.6
img = cv2.resize(img, (0, 0), fx=zoom, fy=zoom)
cv2.imshow('img', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [54]:
print(sum(sum(main_grid,[]))) #number of holds

5
